In [1]:
from glob import glob

import os
import pandas as pd
import numpy as np
import json
from pathlib import Path
import torch
import pickle

In [2]:
order = [
    "DeepFool", "BIM", "CW", "Low_CW", 
    "FAB", "FGSM", "PGD", "Low_PGD1", "Low_PGD2", 
    "AutoAttack", "Square"
]

In [43]:
dataname = 'SVHN'
modelname = 'vgg19'

# Adversarial Attack

In [44]:
del_keys = ['clean acc','noise acc','clean dwt acc']

attack_results = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/results.json')
adv_df = pd.DataFrame()

for p in attack_results:
    r = json.load(open(p,'r'))
    
    for k in del_keys:
        del r[k]
    
    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    adv_df = pd.concat([adv_df, r], axis=1)
    
adv_df = ((adv_df.T.loc[order]) * 100).round(2)

adv_df.columns = ['Adv Acc(%)','Adv Acc(%) DWT']

## Successed Images

In [45]:
succ_list = glob(f'./results/{dataname}/saved_adv_samples/{modelname}/*/successed_images.pkl')

In [46]:
for p in succ_list:
    with open(p,'rb') as f:
        nb_imgs = pickle.load(f)['clean'].shape[0]
        adv_method = os.path.basename(os.path.dirname(p))
        adv_df.loc[adv_method,'# Success Images'] = nb_imgs

adv_df['# Success Images'] = adv_df['# Success Images'].astype(int)

In [47]:
print(adv_df.to_markdown())

|            |   Adv Acc(%) |   Adv Acc(%) DWT |   # Success Images |
|:-----------|-------------:|-----------------:|-------------------:|
| DeepFool   |         1.4  |            54.58 |              23937 |
| BIM        |         1.05 |            29.9  |              24015 |
| CW         |        15.68 |            69.78 |              20208 |
| Low_CW     |        74.88 |            92.11 |               4829 |
| FAB        |         0.74 |            90.69 |              24116 |
| FGSM       |        20.19 |            44.18 |              19039 |
| PGD        |         1.26 |            31.61 |              23982 |
| Low_PGD1   |        81.78 |            91.85 |               3085 |
| Low_PGD2   |        52.86 |            83.1  |              10551 |
| AutoAttack |         0.44 |            34.92 |              24170 |
| Square     |         2.98 |            82.6  |              23531 |


# Detection

## Known Attack

In [48]:
del_keys = ['TNR','AUIN','AUOUT']

known_results = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/result.json')
known_df = pd.DataFrame()

for p in known_results:
    r = json.load(open(p,'r'))['test']

    for k in del_keys:
        del r[k]

    r = pd.Series(
        r,
        name = os.path.basename(os.path.dirname(p))
    )
    
    known_df = pd.concat([known_df, r], axis=1)

known_df = ((known_df.T.loc[order]) * 100).round(2)

known_df.columns = ['AUROC(%)','Detection Acc(%)']

## Train and Test images

In [49]:
train_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/train.pt')
dev_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/dev.pt')
test_det = glob(f'./results/{dataname}/known_attack_results/{modelname}/*/test.pt')

In [50]:
for train_p, dev_p, test_p in zip(train_det, dev_det, test_det):
    nb_train = torch.load(train_p)['logits'].size(0)
    nb_dev = torch.load(dev_p)['logits'].size(0)    
    nb_test = torch.load(test_p)['logits'].size(0)
    
    adv_method = os.path.basename(os.path.dirname(train_p))
    
    known_df.loc[adv_method, '#(train, dev, test)'] = f"({nb_train}, {nb_dev}, {nb_test})"

In [51]:
print(known_df.to_markdown())

|            |   AUROC(%) |   Detection Acc(%) | #(train, dev, test)   |
|:-----------|-----------:|-------------------:|:----------------------|
| DeepFool   |      95.4  |              89.52 | (18605, 3100, 9304)   |
| BIM        |      98.96 |              95.2  | (24104, 4017, 12055)  |
| CW         |      93.56 |              87.75 | (14674, 2445, 7339)   |
| Low_CW     |      90.12 |              84.41 | (3358, 559, 1683)     |
| FAB        |      96.75 |              92.39 | (15473, 2578, 7739)   |
| FGSM       |      92.78 |              86.19 | (15777, 2628, 7891)   |
| PGD        |      98.9  |              94.87 | (23855, 3975, 11931)  |
| Low_PGD1   |      78.78 |              72.43 | (2203, 365, 1106)     |
| Low_PGD2   |      88.59 |              81.52 | (7886, 1313, 3948)    |
| AutoAttack |      98.94 |              94.77 | (23558, 3925, 11782)  |
| Square     |      98.3  |              94.83 | (15747, 2624, 7876)   |


## Transfer Attack

In [52]:
transfer_df = pd.read_csv(f'./results/{dataname}/transfer_attack_results/{modelname}/transfer_results.csv',index_col=0)
print(transfer_df.loc[order, order].to_markdown())

|            |   DeepFool |   BIM |    CW |   Low_CW |   FAB |   FGSM |   PGD |   Low_PGD1 |   Low_PGD2 |   AutoAttack |   Square |
|:-----------|-----------:|------:|------:|---------:|------:|-------:|------:|-----------:|-----------:|-------------:|---------:|
| DeepFool   |      95.4  | 35.78 | 95.32 |    94.09 | 94.48 |  95.07 | 33.91 |      90.72 |      82.15 |        39.71 |    93.9  |
| BIM        |      32.3  | 98.96 | 31.01 |    22.1  | 34.79 |  31.1  | 99.05 |      55.46 |      91.36 |        99.06 |    16.17 |
| CW         |      93.02 | 40.19 | 93.56 |    92.91 | 92.86 |  93.21 | 38.05 |      90.77 |      85.13 |        44.09 |    92.19 |
| Low_CW     |      86.67 | 47.91 | 87.49 |    90.12 | 86.96 |  84.54 | 44.6  |      87.35 |      78.68 |        50.73 |    89.86 |
| FAB        |      95.67 | 40.36 | 95.64 |    95.64 | 96.75 |  95.33 | 38.83 |      91.82 |      83.18 |        44.95 |    96.3  |
| FGSM       |      92.08 | 38.91 | 92.23 |    90.46 | 91.06 |  92.78 | 36.3